##Libraries&Load Data


In [1]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
import joblib
import pickle
from scipy.optimize import minimize, LinearConstraint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/Tesi/data/Dati/'
data_load_path = path + 'data/load/data_load.xlsx'
data_prod_path = path + 'data/production/data_production.xlsx'
data_load = pd.read_excel(data_load_path, decimal=',')
data_prod = pd.read_excel(data_prod_path, decimal=',')
data_load = data_load.set_index("datetime")
data_prod = data_prod.set_index("datetime")

In [4]:
def df_to_X_y(df, window_size):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np) - window_size):
      row = [a for a in df_as_np[i : i + window_size]]
      X.append(row)
      label = df_as_np[i + window_size]
      y.append(label)
  return np.array(X), np.array(y)

In [5]:
win_size = 3
X_load, y_load = df_to_X_y(data_load, win_size)
X_prod, y_prod = df_to_X_y(data_prod, win_size)

y_load = y_load.flatten()
y_prod = y_prod.flatten()

In [6]:
# Load the models

model_load_path = path + 'models/load/gru_model.keras'
model_load_gru = tf.keras.models.load_model(model_load_path)

In [7]:
model_prod_path = path + 'models/production/sarima_model.pkl'
from statsmodels.tsa.arima_model import ARIMAResults
model_prod_sarima = joblib.load(model_prod_path)

In [8]:
# Make predictions using the loaded model

load = data_load.iloc[3:]
load['pred'] = model_load_gru.predict(X_load).flatten()
load

274/274 [==============================] - 1s 2ms/step


<ipython-input-8-c483df6ed651>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  load['pred'] = model_load_gru.predict(X_load).flatten()


,carico,pred
datetime,,
2013-01-01 03:00:00,0.071443,0.117215
2013-01-01 04:00:00,0.071443,0.113994
2013-01-01 05:00:00,0.071443,0.114024
2013-01-01 06:00:00,0.071443,0.114461
2013-01-01 07:00:00,0.071443,0.114461
...,...,...
2013-12-31 19:00:00,0.295625,-0.308975
2013-12-31 20:00:00,0.295625,-0.440229
2013-12-31 21:00:00,0.285771,-0.156215


In [9]:
forecast_steps = 8757
prod = data_prod.iloc[3:]
df_prod = pd.DataFrame()
df_prod['pred'] = model_prod_sarima.forecast(steps=forecast_steps)
df_prod

,pred
2013-11-01 00:00:00,4.847620e-05
2013-11-01 01:00:00,8.287851e-05
2013-11-01 02:00:00,9.614792e-05
2013-11-01 03:00:00,9.660002e-05
2013-11-01 04:00:00,9.000880e-05
...,...
2014-10-31 16:00:00,1.539265e-02
2014-10-31 17:00:00,8.838468e-04
2014-10-31 18:00:00,2.650953e-05
2014-10-31 19:00:00,4.237132e-07


##Creation Data Structure

In [10]:
ESS_storage_max = 0.5
ESS_storage = 0
n_prosumers = 3
n_consumers = 3
prosumers = []
prosumers_c = []
consumers = []
energy_price = 0.174
omega_t = [str(ts) for ts in load.index[:-1]]

In [11]:
# Generation of Producers and Consumers

s_p = 'P'
for i in range(0, n_prosumers):
    r = s_p + str(i+1)
    prosumers.append(r)

s_pc = 'P_C'
for i in range(0, n_prosumers):
    r = s_pc + str(i+1)
    prosumers_c.append(r)

s_c = 'C'
for i in range(0, n_consumers):
    u = s_c + str(i+1)
    consumers.append(u)

prosumers_consumers = prosumers + prosumers_c + consumers + ['tot_energy']
suppliers = ['ESS', 'Grid', 'tot_load']
ESS = 0
Grid = 0

In [12]:
# Creation Dataframe M for time t (M[t])

zeros_array = np.zeros((len(suppliers), len(prosumers_consumers)))
M = pd.DataFrame(zeros_array, index=suppliers, columns=prosumers_consumers)
M

,P1,P2,P3,P_C1,P_C2,P_C3,C1,C2,C3,tot_energy
ESS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tot_load,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##Data Generation (1 hour x 8760 hours)

In [13]:
# Data Generation (1 hour) * 8760 (hour in a year)
#es. 2013-01-01 12:00:00

In [ ]:
# Generation of data

def create_M(t, ESS_storage):
    p = prod.loc[t, 'produzionePV']
    l = load.loc[t, 'pred']

    tot_load = []
    tot_prod = []

    for i in range(0, n_prosumers):
        s_pi = s_p + str(i+1)

        # if there is production
        if (p > 0):
          # put energy in the ESS (value predicted + deviation)
          deviation = random.uniform(-0.02, 0.02)
          pi = p + deviation
          tot_prod.append(pi)
          # energy in storage
          ESS_storage = sum(M[:-1].loc['ESS'])
          # if the ESS is not full
          if ESS_storage < ESS_storage_max :
            # check if the energy to add is less than the minimum space available
              if (ESS_storage_max -  ESS_storage) > pi :
                M[s_pi].loc['ESS'] += pi
              else:
                # the space available is not enough, so put a part in the ESS and the rest sold to the grid
                M[s_pi].loc['ESS'] = ESS_storage_max - ESS_storage
                M[s_pi].loc['Grid'] += pi - (ESS_storage_max - ESS_storage)
          else:
              M[s_pi].loc['Grid'] += pi - (ESS_storage_max - ESS_storage)
        else:
          tot_prod.append(0)


    for i in range(0, n_prosumers):
        s_pci = s_pc + str(i+1)

        deviation = random.uniform(-0.02, 0.02)
        ci = l + deviation
        # if there is enough energy in the ESS1
        if ESS_storage > 0:
            if (ESS_storage - ci) > 0 or ESS_storage > 0:
                ESS_storage -= ci
                M[s_pci].loc['ESS'] = ci
        else:
            M[s_pci].loc['Grid'] = ci - ESS_storage


    for i in range(0, n_consumers):
        s_ci = s_c + str(i+1)

        deviation = random.uniform(-0.02, 0.02)
        ci = l + deviation
        # if there is enough energy in the ESS2
        if ESS_storage > 0:
            if (ESS_storage - ci) > 0:
                ESS_storage -= ci
                M[s_ci].loc['ESS'] = ci
        else:
            M[s_ci].loc['Grid'] = ci - ESS_storage

    M['tot_energy'].loc['ESS'] = ESS_storage
    M['tot_energy'].loc['Grid'] = sum(M.loc['Grid'].iloc[:-1])
    return M

In [ ]:
dt_M = {}
dt_M[omega_t[0]] = create_M(omega_t[0], 0)

for t in range(1, len(omega_t)):
    # ESS storage of the previous hour
    df = dt_M[omega_t[t-1]]
    ESS_storage = df['tot_energy'].loc['ESS']
    dt_M[omega_t[t]] = create_M(omega_t[t], ESS_storage)

In [ ]:
dt_M

{'2013-01-01 03:00:00':                     P1            P2            P3      P_C1      P_C2  \
 ESS           0.101421      0.075288      0.061995  0.057252  0.079347   
 Grid      64433.218262  64433.471177  64433.468922  0.144141  0.152310   
 tot_load      0.000000      0.000000      0.000000  0.000000  0.000000   
 
               P_C3        C1        C2        C3     tot_energy  
 ESS       0.059747  0.092727  0.086269  0.061656      65.780069  
 Grid      0.141531  0.159228  0.148458  0.157224  193301.061253  
 tot_load  0.000000  0.000000  0.000000  0.000000       0.000000  ,
 '2013-01-01 04:00:00':                     P1            P2            P3      P_C1      P_C2  \
 ESS           0.101421      0.075288      0.061995  0.057252  0.079347   
 Grid      64433.218262  64433.471177  64433.468922  0.144141  0.152310   
 tot_load      0.000000      0.000000      0.000000  0.000000  0.000000   
 
               P_C3        C1        C2        C3     tot_energy  
 ESS       0.0

In [ ]:
# Save to Excel file
#dt_M.to_excel(index=True)

## Optimization

### Variables


In [ ]:
l = [0] * (n_prosumers + n_consumers)
l_2 = [0] * n_prosumers

omega_t = [str(ts) for ts in load.index[:-1]]
omega_alpha = prosumers + prosumers_c + consumers

G_th = 0                  # Energia importata dal Grid da h in t
E_th = 0                  # Energia esportata nel Grid da h in t
X_p2p_th = 0              # Energia esportata nell'ESS da h in t
I_p2p_th = 0              # Energia importata dall'ESS da h in t

D_th = 1                  # potenza di scarica dell’unità di accumulo connessa ad h in t
C_th = 1                  # potenza di carica dell’unità di accumulo connessa ad h in t
dem_th = 0                # domanda complessiva di consumo elettrico di h in t
res_th = 0                # produzione complessiva delle RES connesse ad h in t

c_spot = energy_price     # prezzo orario spot [€/kWh] del mercato locale in t
C_LEM = energy_price      # prezzo orario [€/kWh] del mercato locale in t
C_sell = energy_price     # prezzo di vendita orario del kWh in rete [€/kWh] in t.

res_pv_th_tot = 0
dem_th_tot = 0

variables = []

In [ ]:
# Calculate variables(t) from dt_M

def calculate_variables(dt_M):
    G_th_tot = []
    E_th_tot = [[]]
    X_p2p_th_tot = [[]]
    I_p2p_th_tot = [[]]
    # Prosumers
    for h in range(n_prosumers):
        G_h = []
        E_h = []
        X_p2p_h = []
        I_p2p_h = []
        for t in range(len(omega_t)):
            M = dt_M[omega_t[t]]

            G_h.append(M[omega_alpha[h]].loc['Grid'])
            E_h.append(0)
            X_p2p_h.append(M[omega_alpha[h]].loc['ESS'])
            I_p2p_h.append(0)

        G_th_tot.append(G_h)
        E_th_tot.append(E_h)
        X_p2p_th_tot.append(X_p2p_h)
        I_p2p_th_tot.append(I_p2p_h)

    # Prosumers_Consumers
    for h in range(n_prosumers, n_prosumers+n_prosumers):
        for t in range(len(omega_t)):
            M = dt_M[omega_t[t]]

            G_h.append(0)
            E_h.append(M[omega_alpha[h]].loc['Grid'])
            X_p2p_h.append(0)
            I_p2p_h.append(M[omega_alpha[h]].loc['ESS'])

        G_th_tot.append(G_h)
        E_th_tot.append(E_h)
        X_p2p_th_tot.append(X_p2p_h)
        I_p2p_th_tot.append(I_p2p_h)

        # Consumers
    for h in range(n_prosumers+n_prosumers, n_prosumers+n_prosumers+n_consumers):
        for t in range(len(omega_t)):
            M = dt_M[omega_t[t]]

            G_h.append(0)
            E_h.append(M[omega_alpha[h]].loc['Grid'])
            X_p2p_h.append(0)
            I_p2p_h.append(M[omega_alpha[h]].loc['ESS'])

        G_th_tot.append(G_h)
        E_th_tot.append(E_h)
        X_p2p_th_tot.append(X_p2p_h)
        I_p2p_th_tot.append(I_p2p_h)

    a = sum(sum(row) for row in G_th_tot)
    b = sum(sum(row) for row in E_th_tot)
    c = sum(sum(row) for row in X_p2p_th_tot)
    d = sum(sum(row) for row in I_p2p_th_tot)
    res_pv_th_tot = a + c
    dem_th_tot = b + d
    return [a, d, b, c, res_pv_th_tot, dem_th_tot]

In [ ]:
variables = calculate_variables(dt_M)

In [ ]:
variables

[5077612909.902262,
 26784.432636011516,
 55340.05606342331,
 5347.092411550748,
 5077618256.994674,
 82124.48869943482]

### Optimization problem


In [ ]:
def objective_function(variables):
    return (c_spot * variables[0]) + (C_LEM * variables[1]) - (C_sell * variables[2]) - (C_LEM * variables[3])

In [ ]:
# Constraints
def constraint_1(variables):
    return [variables[0], variables[1], variables[2], variables[3], variables[4], variables[5]]
def constraint_2(variables):
  return (variables[0] + variables[1] + variables[4] - (variables[5] + variables[3] + variables[2]))

# variables must be greater than or equal to 0
bounds = ((0, None),  # Bound for G
          (0, None),  # Bound for Ip2p
          (0, None),  # Bound for E
          (0, None),  # Bound for Xp2p
          (0, None),  # Bound for res_pv
          (0, None))  # Bound for dem

In [ ]:
# Initial guess
initial_guess = [1, 1, 1, 1, 1, 1]

# Define constraints
constraints = [{'type': 'eq', 'fun': constraint_1},
               {'type': 'ineq', 'fun': constraint_2}]

# Solve the optimization problem
result = minimize(objective_function, initial_guess, bounds=bounds, constraints=constraints)

In [ ]:
result

# fun : The value of the objective function at the optimal solution
# x : The optimal values of the variables
# jac : The Jacobian matrix of the constraints at the optimal solution. This indicates the sensitivity of the constraints with respect to changes in the variables.

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.0
       x: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
     nit: 1
     jac: [ 1.740e-01  1.740e-01 -1.740e-01 -1.740e-01  0.000e+00
            0.000e+00]
    nfev: 8
    njev: 1